Download data from street manager and load into anaconda (unless I can work out how to read from hard drive)\
The code block starting DD= needs to refer to the folder and then the Coordinates_Column, Work_Ref_Column and Promoter need to point towards the appropriate columns. \
Everything else should work out of the box. \
Could change the map save name 

In [5]:
from pyproj import Transformer

In [6]:
import folium

In [8]:
import pandas as pd

In [10]:
DD=pd.read_csv("Reinstatements.csv") #DD = Downloaded Data
DD

,Works reference number,Permit reference number,Reinstatement type,Site location description,Site location area,Site location coordinates,Depth,Length,Width,Number of holes,...,Secondary contact number,Secondary contact email,Additional contact,Additional contact number,Additional contact email,USRN,Road/Street,Area,Town,Primary contact
0,LP651S30009046,LP651S30009046-04,Excavation,Poud Farm Carriageway,Carriageway,"Point: 420502.00,102480.00",0.00,0.0,0.0,NaN,...,Not Defined,NaN,CARL JAMES,077678 50383,carl.james@sse.com,26719949,POUND LANE,BURLEY,BURLEY,NaN
1,LP651S30009046,LP651S30009046-04,Excavation,Poud Gate Drive,Footpath,"Point: 420512.00,102483.00",0.00,0.0,0.0,NaN,...,Not Defined,NaN,CARL JAMES,077678 50383,carl.james@sse.com,26719949,POUND LANE,BURLEY,BURLEY,NaN
2,LP651S30009046,LP651S30009046-04,Excavation,Poud Gate Pathway,Footpath,"Point: 420513.00,102484.00",0.00,0.0,0.0,NaN,...,Not Defined,NaN,CARL JAMES,077678 50383,carl.james@sse.com,26719949,POUND LANE,BURLEY,BURLEY,NaN
3,LP651S30009046,LP651S30009046-04,Excavation,Poud Farm Pathway,Footpath,"Point: 420516.00,102483.00",0.00,0.0,0.0,NaN,...,Not Defined,NaN,CARL JAMES,077678 50383,carl.james@sse.com,26719949,POUND LANE,BURLEY,BURLEY,NaN
4,LP651S30009046,LP651S30009046-04,Excavation,Poud Farm Drive,Footpath,"Point: 420544.00,102492.00",0.00,0.0,0.0,NaN,...,Not Defined,NaN,CARL JAMES,077678 50383,carl.james@sse.com,26719949,POUND LANE,BURLEY,BURLEY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7968,MU70500742865-001,MU70500742865-001-01,Excavation,"O/S DES HELYER MOTOR CYCLES, 5,PAMBER HEATH RO...",Carriageway,"Point: 460862.23,161742.92",1.69,1.9,0.9,NaN,...,07747 648554,streetworks.thamesvalley@thameswater.co.uk,NaN,NaN,NaN,2103729,PAMBER HEATH ROAD,NaN,PAMBER HEATH,NaN
7969,MU70500742865-001,MU70500742865-001-01,Excavation,"O/S DES HELYER MOTOR CYCLES, 5,PAMBER HEATH RO...",Carriageway,"Point: 460862.23,161742.92",1.69,1.9,0.9,NaN,...,07747 648554,streetworks.thamesvalley@thameswater.co.uk,NaN,NaN,NaN,2103729,PAMBER HEATH ROAD,NaN,PAMBER HEATH,NaN
7970,MU70500742865-001,MU70500742865-001-01,Excavation,"O/S DES HELYER MOTOR CYCLES, 5,PAMBER HEATH RO...",Carriageway,"Point: 460862.23,161742.92",1.70,3.0,0.5,NaN,...,07747 648554,streetworks.thamesvalley@thameswater.co.uk,NaN,NaN,NaN,2103729,PAMBER HEATH ROAD,NaN,PAMBER HEATH,NaN
7971,LQ002SWAT13759730,LQ002SWAT13759730-02,Excavation,UNIT 1 HYTHE MARINE SHORE RD HYTHE HA,Carriageway,"Point: 442873.50,107552.48",2.00,2.5,1.0,NaN,...,01962 716209,NaN,SOUTHERN WATER (PRODUCTION - 8,01903 272167,NaN,26719994,SHORE ROAD,HYTHE,HYTHE,NRSWA Team


In [12]:
import re

In [14]:
Coordinates_Column = "Site location coordinates" #To be updated depending on which column name holds the coordinates
Work_Ref_Column = "Works reference number"
Promoter = "Promoter"
#add any extra variables that you want to add to the pop ups later on in the next code block

In [15]:
# Step 1: Identify and process Points and LineStrings
def process_coordinates(coord_string):
    if coord_string.startswith("Point:"):
        # Extract single point
        match = re.search(r"Point: ([\d.]+),([\d.]+)", coord_string)
        if match:
            return [(float(match.group(1)), float(match.group(2)))]
    elif coord_string.startswith("LineString:"):
        # Extract multiple points
        matches = re.findall(r"([\d.]+),([\d.]+)", coord_string)
        return [(float(x), float(y)) for x, y in matches]
    return []

# Apply the extraction function
DD['Coordinates'] = DD[Coordinates_Column].apply(process_coordinates)

# Step 2: Convert all coordinates to Latitude/Longitude
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)

def convert_coordinates(coords):
    return [transformer.transform(easting, northing) for easting, northing in coords]

DD['LatLon'] = DD['Coordinates'].apply(convert_coordinates)

# Step 3: Extract the first entry for map centring
first_entry = DD[Coordinates_Column].iloc[0]  # Get the first value in the column
first_coordinates = process_coordinates(first_entry)  # Process the coordinates
first_latlon = transformer.transform(first_coordinates[0][0], first_coordinates[0][1])  # Convert to lat/lon
map_centre = [first_latlon[1], first_latlon[0]]  # Latitude, Longitude

# Step 4: Plot on a Map using Folium
map_ = folium.Map(location=map_centre, zoom_start=16)  # Centre the map on the first coordinate

for _, row in DD.iterrows():
    lat_lon = row['LatLon']
    # Construct the pop-up content
    popup_content = (
        f"Works Reference Number: {row[Work_Ref_Column]}<br>"
        f"Promoter: {row[Promoter]}" #can add extra pop up content here - but reference them by name in the earlier cells
    )
    
    # Properly indent the if-else block inside the loop
    if len(lat_lon) == 1:
        # Add marker for a single point with a custom icon
        folium.Marker(
            location=[lat_lon[0][1], lat_lon[0][0]],  # Latitude, Longitude
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(icon="fa-person-digging", prefix="fa")  # Custom FontAwesome icon
        ).add_to(map_)
    else:
        # Add polyline for a LineString
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in lat_lon],
            color="blue",
            weight=2.5,
            opacity=0.8,
            popup=folium.Popup(popup_content, max_width=300)
        ).add_to(map_)

# Save the map to an HTML file
map_.save('map.html') #purhaps change the name each save so that the HTML file doesn't just keep getting saved over. 
map_

ValueError: Locations is empty.